In [10]:
import os, sys, json
import scipy.stats


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("yelp_review_full")

In [5]:
dataset_test = dataset["test"]

In [9]:
def test_significance(df_train, metric1, metric2):
    correlation, p_value = scipy.stats.spearmanr(df_train[metric1], df_train[metric2])
    print(f'Correlation between {metric1} and {metric2}')
    print(f"Correlation coefficient: {correlation}")
    print(f"P-value: {p_value}")
    significant = False
    if p_value < 0.05:
        significant = True 
    return {
                    "metric1": metric1,
                    "metric2": metric2,
                    "correlation": correlation, 
                    "p-value": p_value, 
                    "signficant": significant
                }

In [7]:
# convert to json
data = []
for i in range(len(dataset_test)):
    data.append({"text": dataset_test[i]["text"], "gt_label": dataset_test[i]["label"]})

In [11]:
data_df = {x['text']: x for x in data}

In [12]:
path_predicted_labels = "/home/anmola/assignments_hw/sentiment-cartography/v2_dir_512_large/inference_results.json"

In [14]:
with open(path_predicted_labels, "r") as f:
    predicted_df = json.load(f)

In [39]:
for elem in predicted_df:
    review_text = elem["review"]
    if review_text not in data_df:
        print(review_text)

        # assert(review_text in data_df)
        break
    data_df[review_text]["multiclass_predicted_label"] = elem["predicted_label"]

It's hard to complain about this place given the price I got it for! \n**Warning** This is a long review, there is a lot to break down.\n\nB -- S\n-- Grandparents have an Interval International timeshare so they booked this place for a week for me\n-- 2 br suite really means 2 adjointed rooms (1 being a jr suite & the other being a regular suit w/ patio)\n-- Located about 10 mins from The Las Vegas Strip\n-- Plenty of free parking and shaded spots\n-- 24 hour pool, say what! BBQ grills available for use\n-- Exercise room\n-- Free shuttle\n-- Paid wireless $10 - $30 (I forget the exact price for how much)\n\nOK, so this place isn't the \"gem of Las Vegas.\" Let's just put that out there! For what it is, it's actually a decent place. My Grammie got me a 2 bedroom suite for a week for under $400. If you can find a place to sleep 6 - 10 people for that amount, let me know! And if it's in old town Las Vegas, no thank you. You can see the sign as you drive down Tropicana, just hang a left an

In [40]:
len(review_text)

2033

In [41]:
len(key_now)

3526

True

In [20]:
with open("test_results.json", "w") as f:
    json.dump(data_df, f, indent=1)

In [37]:
key_now = None
for key in data_df:
    # print()
    if "hard to complain about this place given the price" in key:
        key_now = key
        print(data_df[key])
        break
    # break

{'text': 'It\'s hard to complain about this place given the price I got it for! \\n**Warning** This is a long review, there is a lot to break down.\\n\\nB -- S\\n-- Grandparents have an Interval International timeshare so they booked this place for a week for me\\n-- 2 br suite really means 2 adjointed rooms (1 being a jr suite & the other being a regular suit w/ patio)\\n-- Located about 10 mins from The Las Vegas Strip\\n-- Plenty of free parking and shaded spots\\n-- 24 hour pool, say what! BBQ grills available for use\\n-- Exercise room\\n-- Free shuttle\\n-- Paid wireless $10 - $30 (I forget the exact price for how much)\\n\\nOK, so this place isn\'t the \\"gem of Las Vegas.\\" Let\'s just put that out there! For what it is, it\'s actually a decent place. My Grammie got me a 2 bedroom suite for a week for under $400. If you can find a place to sleep 6 - 10 people for that amount, let me know! And if it\'s in old town Las Vegas, no thank you. You can see the sign as you drive down 

In [24]:
list(data_df.keys())[:2]

['I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!',
 "Don't waste your time.  We had two different people come to our house to give us estimates for a deck (one of them the OWNER).  Both times, we never heard from them.  Not a call, not the estimate, nothing."]

In [26]:
all_keys = list(data_df.keys())

In [38]:
print(key_now)

It's hard to complain about this place given the price I got it for! \n**Warning** This is a long review, there is a lot to break down.\n\nB -- S\n-- Grandparents have an Interval International timeshare so they booked this place for a week for me\n-- 2 br suite really means 2 adjointed rooms (1 being a jr suite & the other being a regular suit w/ patio)\n-- Located about 10 mins from The Las Vegas Strip\n-- Plenty of free parking and shaded spots\n-- 24 hour pool, say what! BBQ grills available for use\n-- Exercise room\n-- Free shuttle\n-- Paid wireless $10 - $30 (I forget the exact price for how much)\n\nOK, so this place isn't the \"gem of Las Vegas.\" Let's just put that out there! For what it is, it's actually a decent place. My Grammie got me a 2 bedroom suite for a week for under $400. If you can find a place to sleep 6 - 10 people for that amount, let me know! And if it's in old town Las Vegas, no thank you. You can see the sign as you drive down Tropicana, just hang a left an

In [27]:
all_keys[700:720]

['This is a NYC restaurant. That is a compliment and a complaint. There is no question the food is outstanding. Arguably the best in all of Charlotte and deserving of national acclaim. \\n\\nHere is my complaint. Have you ever dined somewhere that it spectacular and the place knows its spectacular? I have experienced this in NYC. Sort of like the honor is all mine and I should be grateful they have allowed me to dine there. Maybe it was just my experience, but there was just an air of pretentiousness. \\n\\nSecond. The place is way too small. This means they have to flip the tables quickly to turn a profit. I was there with a party of four. We ordered a bottle of wine and drinks. The beverages came and we had to order. We were not allowed to order our appetizers while we perused the entr\\u00e9e menu drank and chatted. Nope we had to order appetizers and entrees together. The waitress saw the incredulous look on my face and explained that the entrees take a while. \\n\\nWell the entree